# 0. Introduction

This jupyter notebook is used to implement our models.

Import the packages:

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.layers import *
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import img_to_array
import glob
import cv2
import random
from sklearn.model_selection import train_test_split

Set some parameters:

In [ ]:
batch_size = 128
num_classes = 10
epochs = 100
img_rows, img_cols = 28 , 28
input_shape = (img_rows, img_cols, 1)

# 1. Load Data

In this part, we would show how to load the seven datasets used in our study.

## 1.1 Loading Kannada-MNIST and Dig-MNIST

We first show how to load Kannada-MNIST:

In [ ]:
input_fn = './data/kannada/Kannada_MNIST/X_kannada_MNIST_train.npz'
f = np.load(input_fn)
X_train = f['arr_0']
input_fn = './data/kannada/Kannada_MNIST/y_kannada_MNIST_train.npz'
f = np.load(input_fn)
y_train = f['arr_0']
input_fn = './data/kannada/Kannada_MNIST/X_kannada_MNIST_test.npz'
f = np.load(input_fn)
X_test = f['arr_0']
input_fn = './data/kannada/Kannada_MNIST/y_kannada_MNIST_test.npz'
f = np.load(input_fn)
y_test = f['arr_0']
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

Then we show how to load Dig-MNIST:

In [ ]:
input_fn = './data/kannada/Dig_MNIST/X_dig_MNIST.npz'
f = np.load(input_fn)
X_dig = f['arr_0']
input_fn = './data/kannada/Dig_MNIST/y_dig_MNIST.npz'
f = np.load(input_fn)
y_dig = f['arr_0']
print(X_dig.shape)
print(y_dig.shape)

## 1.2 Loading Other Datasets

For other datasets, you only need to change the npz file name and use the following code:

In [ ]:
input_train_test_fn = './data/mnist.npz'
data = np.load(input_train_test_fn)
X_train, X_test, y_train, y_test = data['x_train'], data['x_test'], data['y_train'], data['y_test']
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

# 2. Data Preprocessing

In this part, we show how to calculate edge images and how to implement the data augmentation techniques.

In [ ]:
# split the training data into training and validation sets
X_train, X_vali, y_train, y_vali = train_test_split(X_train, y_train, stratify=y_train, test_size=0.2)
print(X_train.shape)
print(X_vali.shape)
print(y_train.shape)
print(y_vali.shape)

In [ ]:
# Define some functions for data augmentation
def rotateImage(image, angle):
    row, col = image.shape
    center = tuple(np.array([row,col])/2)
    rot_mat = cv2.getRotationMatrix2D(center,angle, 1.0)
    new_image = cv2.warpAffine(image, rot_mat, (col, row))
    return new_image

def translation(img):
    rows, cols = img.shape

    M = np.float32([[1, 0, random.randint(-5,5)], [0, 1, random.randint(-5,5)]])
    dst = cv2.warpAffine(img,M, (cols, rows))
    return dst

In [ ]:
# Processing the training data set with data augmentation techniques
trainImage =[]
edgeTrainImage = []
trainLabel = []
for i in range(X_train.shape[0]):
    img = X_train[i].astype(np.uint8)

    trainImage.append(img)
    
    edgeImage = cv2.Canny(255 - img, 100, 200)
    edgeTrainImage.append(edgeImage)

    trainLabel.append(y_train[i])
    
    # Augment Input Image with Random Rotation
    rotAngel = random.randint(-45, 45)
    imgRot = rotateImage(img, rotAngel)
    trainImage.append(imgRot)
    
    edgeImage = cv2.Canny(255 - imgRot, 100, 200)
    edgeTrainImage.append(edgeImage)

    trainLabel.append(y_train[i])
    
    # Augment Input Image with Block Effect
    blocEffect = cv2.resize(img, (14, 44))
    blocEffect = cv2.resize(blocEffect, (28, 28))
    trainImage.append(blocEffect)

    edgeImage = cv2.Canny(255 - blocEffect, 100, 200)
    edgeTrainImage.append(edgeImage)
    
    trainLabel.append(y_train[i])
    
    # Augment Input Image with Translation
    transImage = translation(img)
    trainImage.append(transImage)

    edgeImage = cv2.Canny(255 - transImage, 100, 200)
    edgeTrainImage.append(edgeImage)
    trainLabel.append(y_train[i])

In [ ]:
# convert class vectors to binary class matrices
trainImage = np.array(trainImage, dtype="float") / 255.0
trainLabel = np.array(trainLabel)
trainImage = trainImage.reshape(trainImage.shape[0], img_rows, img_cols, 1)
edgeTrainImage = np.array(edgeTrainImage, dtype="float") / 255.0
edgeTrainImage = edgeTrainImage.reshape(edgeTrainImage.shape[0], img_rows, img_cols, 1)
print ("checking shape of the input array(s)",trainImage.shape, edgeTrainImage.shape)

In [ ]:
# Processing the testing set
testImage =[]
edgeTestImage = []
testLabel = []
for i in range(X_test.shape[0]):
    img = X_test[i].astype(np.uint8)
    testImage.append(img)
    
    edgeImage = cv2.Canny(255 - img, 100, 200)
    edgeTestImage.append(edgeImage)
    
testLabel = y_test[:]

In [ ]:
# convert class vectors to binary class matrices
testImage = np.array(testImage, dtype="float") / 255.0
testLabel = np.array(testLabel)
testImage = testImage.reshape(testImage.shape[0], img_rows, img_cols, 1)
edgeTestImage = np.array(edgeTestImage, dtype="float") / 255.0
edgeTestImage = edgeTestImage.reshape(edgeTestImage.shape[0], img_rows, img_cols, 1)
print ("checking shape of the input array(s)",testImage.shape, edgeTestImage.shape)

In [ ]:
# Processing the validation set
valiImage =[]
edgeValiImage = []
valiLabel = []
    
for i in range(X_vali.shape[0]):
    img = X_vali[i].astype(np.uint8)
    valiImage.append(img)
        
    edgeImage = cv2.Canny(255 - img, 100, 200)
    edgeValiImage.append(edgeImage)
        
valiLabel = y_vali[:]

In [ ]:
# convert class vectors to binary class matrices
valiImage = np.array(valiImage, dtype="float") / 255.0
valiLabel = np.array(valiLabel)
valiImage = valiImage.reshape(valiImage.shape[0], img_rows, img_cols, 1)
edgeValiImage = np.array(edgeValiImage, dtype="float") / 255.0
edgeValiImage = edgeValiImage.reshape(edgeValiImage.shape[0], img_rows, img_cols, 1)
print ("checking shape of the input array(s)", valiImage.shape, edgeValiImage.shape)

In [ ]:
# build y_train and y_test
y_train = keras.utils.to_categorical(trainLabel, num_classes)
y_test = keras.utils.to_categorical(testLabel, num_classes)
y_vali = keras.utils.to_categorical(valiLabel, num_classes)

# 3 Build models

## 3.1 CNN

In [ ]:
K.set_image_dim_ordering('tf')

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# save weights based on the accuracy on the validation set, remeber to change the save path here
checkpoint = ModelCheckpoint('./trained_model/with-data-augmentation/cnn/mnist/{val_acc:.4f}.hdf5',
                             monitor='val_acc', verbose=1, save_best_only=True, mode='auto')

model.summary()

In [ ]:
#Compile Keras Model 
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
    
model.fit(trainImage, y_train,
          batch_size=batch_size, shuffle=True,
          epochs=epochs,
          callbacks=[checkpoint],
          verbose=2,
          validation_data=(valiImage, y_vali))
score = model.evaluate(testImage, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

## 3.2 EdgeNet

In [ ]:
K.set_image_dim_ordering('tf')

#Input PlaceHolder
inpX = Input(input_shape)
inpY = Input(input_shape)
inputImg = Conv2D(16, kernel_size=3, strides=1, padding='same', activation='relu', name="inputImgConv")(inpX)
inputEdge= Conv2D(16, kernel_size=3, strides=1, padding='same', activation='relu', name="inputEdgeConv")(inpY)
    
inputMerge = Add()([inputImg, inputEdge])
layer1 = Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu', name="convLayer1")(inputMerge)
layer1 = Dropout(0.20, name="convLayer1Dropout")(layer1)
layer2 = Conv2D(32, kernel_size=3, strides=1, padding='same', dilation_rate=(2, 2), activation='relu', name="convLayer2")(layer1)
layer3 = Conv2D(32, kernel_size=3, strides=1, padding='same', dilation_rate=(2, 2), activation='relu', name="convLayer3")(layer2)
layer3 = Dropout(0.25, name="convLayer3Dropout")(layer3)
shortCut = Add(name="edgeShortcut")([inpY, layer3])
    
layer4 = Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu', name="convLayer4")(shortCut)
layer4 = Dropout(0.25, name="convLayer4Dropout")(layer4)
layer4 = AveragePooling2D((2, 2), name="globalPoolLayer")(layer4)
denseLayer = Flatten()(layer4)
denseLayer = Dense(128, activation='relu', name="denseLayer")(denseLayer)
denseLayer = Dropout(0.25, name="denseLayerDropout")(denseLayer)
outputLayer = Dense(num_classes, activation='softmax', name="classifier")(denseLayer)
model = Model([inpX, inpY], outputLayer)
# save weights based on the accuracy on the validation set, remeber to change the save path here
checkpoint = ModelCheckpoint('./trained_model/with-data-augmentation/edgenet/mnist/{val_acc:.4f}.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
# Model Summary
model.summary()

In [ ]:
# Compile Keras Model 
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
    
model.fit([trainImage,edgeTrainImage], y_train,
          batch_size=batch_size, shuffle=True,
          epochs=epochs,
          callbacks=[checkpoint],
          verbose=2,
          validation_data=([valiImage, edgeValiImage], y_vali))
score = model.evaluate([testImage, edgeTestImage], y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

## 3.3 Edge-SiamNet

In [ ]:
K.set_image_dim_ordering('tf')

def create_base_network(input_shape):
    '''Base network to be shared (eq. to feature extraction).
    '''
    input = Input(shape=input_shape)
    # x = Conv2D(16, kernel_size=3, strides=1, padding='same', activation='relu', name="inputImgConv")(input)
    x = Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu', name="convLayer1")(input)
    x = Dropout(0.20, name="convLayer1Dropout")(x)
    x = Conv2D(32, kernel_size=3, strides=1, padding='same', dilation_rate=(2, 2), activation='relu', name="convLayer2")(x)
    x = Conv2D(32, kernel_size=3, strides=1, padding='same', dilation_rate=(2, 2), activation='relu', name="convLayer3")(x)
    x = Dropout(0.25, name="convLayer3Dropout")(x)
    return Model(input, x)


input_a = Input(input_shape)
input_b = Input(input_shape)

inputImg = Conv2D(16, kernel_size=3, strides=1, padding='same', activation='relu', name="inputImgConv")(input_a)
inputEdge= Conv2D(16, kernel_size=3, strides=1, padding='same', activation='relu', name="inputEdgeConv")(input_b)

# inputMerge = Add()([inputImg, inputEdge])

# network definition
base_network = create_base_network((28, 28, 16))

# because we re-use the same instance `base_network`,
# the weights of the network
# will be shared across the two branches
processed_a = base_network(inputImg)
processed_b = base_network(inputEdge)

inputMerge = Add()([processed_a, processed_b])
layer4 = Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu', name="convLayer4")(inputMerge)
layer4 = Dropout(0.25, name="convLayer4Dropout")(layer4)
layer4 = AveragePooling2D((2, 2), name="globalPoolLayer")(layer4)
denseLayer = Flatten()(layer4)
denseLayer = Dense(128, activation='relu', name="denseLayer")(denseLayer)
denseLayer = Dropout(0.25, name="denseLayerDropout")(denseLayer)
outputLayer = Dense(num_classes, activation='softmax', name="classifier")(denseLayer)
model = Model([input_a, input_b], outputLayer)
# save weights based on the accuracy on the validation set, remeber to change the save path here
checkpoint = ModelCheckpoint('./trained_model/with-data-augmentation/edge-siamnet/mnist/{val_acc:.4f}.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
# Model Summary
model.summary()

In [ ]:
# Compile Keras Model 
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
model.fit([trainImage, edgeTrainImage], y_train,
          batch_size=batch_size, shuffle=True,
          epochs=epochs,
          callbacks=[checkpoint],
          verbose=2,
          validation_data=([valiImage,edgeValiImage], y_vali))
score = model.evaluate([testImage, edgeTestImage], y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

## 3.4 Edge-TripleNet

In [ ]:
K.set_image_dim_ordering('tf')

def create_base_network(input_shape):
    '''Base network to be shared (eq. to feature extraction).
    '''
    input = Input(shape=input_shape)
    # x = Conv2D(16, kernel_size=3, strides=1, padding='same', activation='relu', name="inputImgConv")(input)
    x = Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu', name="convLayer1")(input)
    x = Dropout(0.20, name="convLayer1Dropout")(x)
    x = Conv2D(32, kernel_size=3, strides=1, padding='same', dilation_rate=(2, 2), activation='relu', name="convLayer2")(x)
    x = Conv2D(32, kernel_size=3, strides=1, padding='same', dilation_rate=(2, 2), activation='relu', name="convLayer3")(x)
    x = Dropout(0.25, name="convLayer3Dropout")(x)
    return Model(input, x)


input_a = Input(input_shape)
input_b = Input(input_shape)

inputImg = Conv2D(16, kernel_size=3, strides=1, padding='same', activation='relu', name="inputImgConv")(input_a)
inputEdge= Conv2D(16, kernel_size=3, strides=1, padding='same', activation='relu', name="inputEdgeConv")(input_b)

inputMerge = Add()([inputImg, inputEdge])

# network definition
base_network = create_base_network((28, 28, 16))

# because we re-use the same instance `base_network`,
# the weights of the network
# will be shared across the two branches
processed_a = base_network(inputImg)
processed_b = base_network(inputEdge)
processed_c = base_network(inputMerge)

inputMerge = Add()([processed_a, processed_b, processed_c])
layer4 = Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu', name="convLayer4")(inputMerge)
layer4 = Dropout(0.25, name="convLayer4Dropout")(layer4)
layer4 = AveragePooling2D((2, 2), name="globalPoolLayer")(layer4)
denseLayer = Flatten()(layer4)
denseLayer = Dense(128, activation='relu', name="denseLayer")(denseLayer)
denseLayer = Dropout(0.25, name="denseLayerDropout")(denseLayer)
outputLayer = Dense(num_classes, activation='softmax', name="classifier")(denseLayer)
model = Model([input_a, input_b], outputLayer)
# save weights based on the accuracy on the validation set, remeber to change the save path here
checkpoint = ModelCheckpoint('./trained_model/with-data-augmentation/edge-triplenet/mnist/{val_acc:.4f}.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
#Model Summary
model.summary()

In [ ]:
# Compile Keras Model 
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
model.fit([trainImage, edgeTrainImage], y_train,
          batch_size=batch_size, shuffle=True,
          epochs=epochs,
          callbacks=[checkpoint],
          verbose=2,
          validation_data=([valiImage,edgeValiImage], y_vali))
score = model.evaluate([testImage, edgeTestImage], y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# 4 Use trained models

In this part, we show how to use the trained models. We build models as the same as the previous section, but then we load the weights directly.

## 4.1 CNN

In [ ]:
K.set_image_dim_ordering('tf')

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

In [ ]:
# you need to change the number here
best_weight = '0.9951'

# Compile Keras Model 
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
    
# load weights
model.load_weights('./trained_model/with-data-augmentation/cnn/mnist/%s.hdf5' % best_weight)

score = model.evaluate(testImage, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

## 4.2 EdgeNet

In [ ]:
K.set_image_dim_ordering('tf')

#Input PlaceHolder
inpX = Input(input_shape)
inpY = Input(input_shape)
inputImg = Conv2D(16, kernel_size=3, strides=1, padding='same', activation='relu', name="inputImgConv")(inpX)
inputEdge= Conv2D(16, kernel_size=3, strides=1, padding='same', activation='relu', name="inputEdgeConv")(inpY)
    
inputMerge = Add()([inputImg, inputEdge])
layer1 = Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu', name="convLayer1")(inputMerge)
layer1 = Dropout(0.20, name="convLayer1Dropout")(layer1)
layer2 = Conv2D(32, kernel_size=3, strides=1, padding='same', dilation_rate=(2, 2), activation='relu', name="convLayer2")(layer1)
layer3 = Conv2D(32, kernel_size=3, strides=1, padding='same', dilation_rate=(2, 2), activation='relu', name="convLayer3")(layer2)
layer3 = Dropout(0.25, name="convLayer3Dropout")(layer3)
shortCut = Add(name="edgeShortcut")([inpY, layer3])
    
layer4 = Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu', name="convLayer4")(shortCut)
layer4 = Dropout(0.25, name="convLayer4Dropout")(layer4)
layer4 = AveragePooling2D((2, 2), name="globalPoolLayer")(layer4)
denseLayer = Flatten()(layer4)
denseLayer = Dense(128, activation='relu', name="denseLayer")(denseLayer)
denseLayer = Dropout(0.25, name="denseLayerDropout")(denseLayer)
outputLayer = Dense(num_classes, activation='softmax', name="classifier")(denseLayer)
model = Model([inpX, inpY], outputLayer)

# Model Summary
model.summary()

In [ ]:
# you need to change the number here
best_weight = '0.9956'

# Compile Keras Model 
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
    
# load weights
model.load_weights('./trained_model/with-data-augmentation/edgenet/mnist/%s.hdf5' % best_weight)

score = model.evaluate([testImage, edgeTestImage], y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

## 4.3 Edge-SiamNet

In [ ]:
K.set_image_dim_ordering('tf')

def create_base_network(input_shape):
    '''Base network to be shared (eq. to feature extraction).
    '''
    input = Input(shape=input_shape)
    # x = Conv2D(16, kernel_size=3, strides=1, padding='same', activation='relu', name="inputImgConv")(input)
    x = Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu', name="convLayer1")(input)
    x = Dropout(0.20, name="convLayer1Dropout")(x)
    x = Conv2D(32, kernel_size=3, strides=1, padding='same', dilation_rate=(2, 2), activation='relu', name="convLayer2")(x)
    x = Conv2D(32, kernel_size=3, strides=1, padding='same', dilation_rate=(2, 2), activation='relu', name="convLayer3")(x)
    x = Dropout(0.25, name="convLayer3Dropout")(x)
    return Model(input, x)


input_a = Input(input_shape)
input_b = Input(input_shape)

inputImg = Conv2D(16, kernel_size=3, strides=1, padding='same', activation='relu', name="inputImgConv")(input_a)
inputEdge= Conv2D(16, kernel_size=3, strides=1, padding='same', activation='relu', name="inputEdgeConv")(input_b)

# inputMerge = Add()([inputImg, inputEdge])

# network definition
base_network = create_base_network((28, 28, 16))

# because we re-use the same instance `base_network`,
# the weights of the network
# will be shared across the two branches
processed_a = base_network(inputImg)
processed_b = base_network(inputEdge)

inputMerge = Add()([processed_a, processed_b])
layer4 = Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu', name="convLayer4")(inputMerge)
layer4 = Dropout(0.25, name="convLayer4Dropout")(layer4)
layer4 = AveragePooling2D((2, 2), name="globalPoolLayer")(layer4)
denseLayer = Flatten()(layer4)
denseLayer = Dense(128, activation='relu', name="denseLayer")(denseLayer)
denseLayer = Dropout(0.25, name="denseLayerDropout")(denseLayer)
outputLayer = Dense(num_classes, activation='softmax', name="classifier")(denseLayer)
model = Model([input_a, input_b], outputLayer)

# Model Summary
model.summary()

In [ ]:
# you need to change the number here
best_weight = '0.9957'

# Compile Keras Model 
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
    
# load weights
model.load_weights('./trained_model/with-data-augmentation/edge-siamnet/mnist/%s.hdf5' % best_weight)

score = model.evaluate([testImage, edgeTestImage], y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

## 4.4 Edge-TripleNet

In [ ]:
K.set_image_dim_ordering('tf')

def create_base_network(input_shape):
    '''Base network to be shared (eq. to feature extraction).
    '''
    input = Input(shape=input_shape)
    # x = Conv2D(16, kernel_size=3, strides=1, padding='same', activation='relu', name="inputImgConv")(input)
    x = Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu', name="convLayer1")(input)
    x = Dropout(0.20, name="convLayer1Dropout")(x)
    x = Conv2D(32, kernel_size=3, strides=1, padding='same', dilation_rate=(2, 2), activation='relu', name="convLayer2")(x)
    x = Conv2D(32, kernel_size=3, strides=1, padding='same', dilation_rate=(2, 2), activation='relu', name="convLayer3")(x)
    x = Dropout(0.25, name="convLayer3Dropout")(x)
    return Model(input, x)


input_a = Input(input_shape)
input_b = Input(input_shape)

inputImg = Conv2D(16, kernel_size=3, strides=1, padding='same', activation='relu', name="inputImgConv")(input_a)
inputEdge= Conv2D(16, kernel_size=3, strides=1, padding='same', activation='relu', name="inputEdgeConv")(input_b)

inputMerge = Add()([inputImg, inputEdge])

# network definition
base_network = create_base_network((28, 28, 16))

# because we re-use the same instance `base_network`,
# the weights of the network
# will be shared across the two branches
processed_a = base_network(inputImg)
processed_b = base_network(inputEdge)
processed_c = base_network(inputMerge)

inputMerge = Add()([processed_a, processed_b, processed_c])
layer4 = Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu', name="convLayer4")(inputMerge)
layer4 = Dropout(0.25, name="convLayer4Dropout")(layer4)
layer4 = AveragePooling2D((2, 2), name="globalPoolLayer")(layer4)
denseLayer = Flatten()(layer4)
denseLayer = Dense(128, activation='relu', name="denseLayer")(denseLayer)
denseLayer = Dropout(0.25, name="denseLayerDropout")(denseLayer)
outputLayer = Dense(num_classes, activation='softmax', name="classifier")(denseLayer)
model = Model([input_a, input_b], outputLayer)

# Model Summary
model.summary()

In [ ]:
# you need to change the number here
best_weight = '0.9957'

# Compile Keras Model 
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
    
# load weights
model.load_weights('./trained_model/with-data-augmentation/edge-triplenet/mnist/%s.hdf5' % best_weight)

score = model.evaluate([testImage, edgeTestImage], y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])